In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
  Cloning https://github.com/huggingface/peft to /tmp/pip-install-_zykaweh/peft_374c7c66d2a144b7829e726fa02e7506
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-install-_zykaweh/peft_374c7c66d2a144b7829e726fa02e7506
  Resolved https://github.com/huggingface/peft to commit 5cdade973eb345b0a8aeda52f3c2fb4199d0a028
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import json

# Charger le fichier JSON
with open('/content/data_code.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
sample = train_dataset.iloc[0]  # Exemple de ligne de votre DataFrame
print(type(sample))  # Vérifiez le type de sample
print(sample)  # Vérifiez la structure des données


<class 'pandas.core.series.Series'>
output         Illustrates the usage of a custom macro to pri...
input          macro_rules! print_with_info { ($val:expr) => ...
instruction    Define a custom macro using macro_rules! and u...
Name: 603, dtype: object


In [2]:
import pandas as pd

data_df = pd.read_json('/content/data_code.json')

In [ ]:


from datasets import load_dataset,Dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)

import numpy as np
import random
import pandas as pd
from IPython.display import display
import evaluate
import logging

import warnings
warnings.filterwarnings("ignore")

torch.cuda.empty_cache()
#_______________________________________________________________________

# Configurer le logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

seed = 69
set_seed(seed)
n_gpus = torch.cuda.device_count()
print(torch.cuda.get_device_name(0))

print(torch.__version__)
print(torch.cuda.is_available())
print("NOMBRE DE GPU : ", n_gpus)

accuracy_metric = evaluate.load("accuracy")

#_______________________________________________________________________

def compute_metrics(eval_pred):
       logits, labels = eval_pred
       # Assuming logits are (batch_size, seq_len, vocab_size)
       predictions = np.argmax(logits, axis=-1)
       predictions = predictions.flatten()
       labels = labels.flatten()
       # Masking out padding predictions and labels for metric calculation, if needed
       valid_indices = labels != -100
       predictions = predictions[valid_indices]
       labels = labels[valid_indices]
       return accuracy_metric.compute(predictions=predictions, references=labels)

#_______________________________________________________________________

def load_model(model_path):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{50000}MB'
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=quantization_config,
        torch_dtype="auto",
        device_map="cuda:0",
        max_memory={i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

#_______________________________________________________________________

dataset = Dataset.from_pandas(data_df)

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

nb_samples = 3
random_indices = random.sample(range(len(dataset)), nb_samples)
samples = []
for idx in random_indices:
    sample = dataset[idx]
    sample_data = {
        'instruction': sample['instruction'],
        'input': sample['input'],
        'output': sample['output']
    }
    samples.append(sample_data)
df = pd.DataFrame(samples)
display(df)

#_______________________________________________________________________

def create_prompt_formats(sample):
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "### Input:"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]
    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt
    return sample

print(create_prompt_formats(dataset[0])["text"])

#_______________________________________________________________________

def get_max_length(model):
    conf = model.config
    max_length = getattr(conf, 'n_positions', None) or getattr(conf, 'max_position_embeddings', None) or getattr(conf, 'seq_length', None) or 1024
    print(max_length)
    return max_length

#_______________________________________________________________________

def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(batch["text"], max_length=max_length, truncation=True)

#_______________________________________________________________________

def preprocess_dataset(tokenizer, max_length, seed, dataset):
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(_preprocessing_function, batched=True, remove_columns=["instruction", "input", "output", "text"])
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    dataset = dataset.shuffle(seed=seed)
    return dataset

#_______________________________________________________________________

def create_peft_config(modules):
    return LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

#_______________________________________________________________________

def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

#_______________________________________________________________________

def print_trainable_parameters(model):
    trainable_params, all_param = 0, 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    print(f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}")

#_______________________________________________________________________

model_path = "stabilityai/stablecode-3b"
model, tokenizer = load_model(model_path)

max_length = get_max_length(model)

# Split the dataset into training and evaluation sets
split_ratio = 0.2
splits = dataset.train_test_split(test_size=split_ratio, seed=seed)
train_dataset = splits['train']
eval_dataset = splits['test']

# Preprocess datasets
train_dataset = preprocess_dataset(tokenizer, max_length, seed, train_dataset)
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, eval_dataset)

#_______________________________________________________________________

# Setup directories upfront
output_dir = "/content/final_checkpoint3"
output_merged_dir = "/content/final_checkpoint_merged3"

# Create directories if not exists
for dir_path in [output_dir, output_merged_dir]:
    os.makedirs(dir_path, exist_ok=True)

#_______________________________________________________________________

def train(model, tokenizer, train_dataset, eval_dataset, output_dir):
    print("Starting the training process.")

    # Enable gradient checkpointing
    # model.gradient_checkpointing_enable()
    # print("Gradient checkpointing enabled.")

    # Prepare model for k-bit training and locate linear modules
    model = prepare_model_for_kbit_training(model)

    # Freeze all layers except the last one
    for name, param in model.named_parameters():
        if "lm_head" not in name:  # Change "lm_head" according to the name of the final layer in your model
            param.requires_grad = False
        else:
            param.requires_grad = True

    # Output the trainable parameters (this should now be only the last layer)
    print_trainable_parameters(model)

    # Obtain PEFT model
    modules = find_all_linear_names(model)
    print(f"Model prepared for k-bit training. Located linear modules: {modules}")
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    print("PEFT model obtained.")

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            max_steps=10,
            learning_rate=1e-5,
            #fp16=True,          --> removed because redundancy with "load in 8bit"
            logging_steps=10,
            output_dir="outputs",
            optim="paged_adamw_8bit",
            weight_decay=0.01,
            eval_strategy="no",
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,
    )
    print("Trainer initialized.")

    model.config.use_cache = False

    # Training
    print(f"BEGINNING TRAINING")
    train_result = trainer.train()
    print(f"Training completed. Metrics: {train_result.metrics}")

    # Evaluate
    print(f"BEGINNING EVALUATION")
    eval_result = trainer.evaluate()
    print(f"Evaluation completed. Results: {eval_result}")

    # Save trained model
    trainer.model.save_pretrained(output_dir)

    # Clear resources
    del model
    del trainer
    torch.cuda.empty_cache()
    print("Training process completed and resources cleaned up.")

#_______________________________________________________________________

# Run training process
train(model, tokenizer, train_dataset, eval_dataset, output_dir)

# Merge weights if needed
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto") # removed torch_dtype=torch.bfloat16 because of redundancy because of load in 8bit
model = model.merge_and_unload()
model.save_pretrained(output_merged_dir, safe_serialization=True)
tokenizer.save_pretrained(output_merged_dir)

NVIDIA A100-SXM4-40GB
2.5.1+cu124
True
NOMBRE DE GPU :  2
Number of prompts: 604
Column names are: ['output', 'input', 'instruction']


,instruction,input,output
0,Custom Types,"let result = x.run(5, 6);",Calls the 'run' method on the 'x' instance wit...
1,,pub fn find_fixed_point(arr: &[i32]) -> Option...,find a fixed point in a given array arguments:...
2,,fn spiral_order(matrix: Vec<Vec<i32>>) -> Vec<...,Returns all elements of a matrix in spiral ord...


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Hello World

### Input:
println!({} days, 31);

### Output:
Prints 31 days using {} as a placeholder for the value (31).

### End


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

16384
Preprocessing dataset...


...

Starting the training process.
all params: 1,526,666,240 || trainable params: 128,778,240 || trainable%: 8.435258252648595
Model prepared for k-bit training. Located linear modules: ['k_proj', 'o_proj', 'up_proj', 'q_proj', 'down_proj', 'gate_proj', 'v_proj']
PEFT model obtained.


max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Trainer initialized.
BEGINNING TRAINING


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: thomas-costa (thomas-costa-efrei). Use `wandb login --relogin` to force relogin


Step,Training Loss


In [3]:
from datasets import load_dataset, Dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

import numpy as np
import random
import pandas as pd
from IPython.display import display
import evaluate
import logging

import warnings
warnings.filterwarnings("ignore")

#_______________________________________________________________________

# Configurer le logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

seed = 69
set_seed(seed)
n_gpus = torch.cuda.device_count()
print(torch.cuda.get_device_name(0))

print(torch.__version__)
print(torch.cuda.is_available())
print("NOMBRE DE GPU : ", n_gpus)

accuracy_metric = evaluate.load("accuracy")

#_______________________________________________________________________

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

#_______________________________________________________________________

def load_model(model_path):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{10000}MB'
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="cuda:0",
        max_memory={i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

#_______________________________________________________________________

dataset = Dataset.from_pandas(data_df)
'''splits = dataset.train_test_split(test_size=split_ratio, seed=42)
dataset = load_dataset(
    'json',
    data_files='http://10.3.220.77:8000/Downloads/projet_llm/llm_project_M2/data/data_code.json',
    split='train'
)'''

print(dataset)

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

nb_samples = 3
random_indices = random.sample(range(len(dataset)), nb_samples)
samples = []
for idx in random_indices:
    sample = dataset[idx]
    sample_data = {
        'instruction': sample['instruction'],
        'input': sample['input'],
        'output': sample['output']
    }
    samples.append(sample_data)
df = pd.DataFrame(samples)
display(df)

#_______________________________________________________________________

def create_prompt_formats(sample):
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "### Input:"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]
    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt
    return sample

print(create_prompt_formats(dataset[0])["text"])

#_______________________________________________________________________

def get_max_length(model):
    conf = model.config
    max_length = getattr(conf, 'n_positions', None) or getattr(conf, 'max_position_embeddings', None) or getattr(conf, 'seq_length', None) or 1024
    print(max_length)
    return max_length

#_______________________________________________________________________

def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(batch["text"], max_length=max_length, truncation=True)

#_______________________________________________________________________

def preprocess_dataset(tokenizer, max_length, seed, dataset):
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(_preprocessing_function, batched=True, remove_columns=["instruction", "input", "output", "text"])
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    dataset = dataset.shuffle(seed=seed)
    return dataset

#_______________________________________________________________________

def create_peft_config(modules):
    return LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

#_______________________________________________________________________

def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

#_______________________________________________________________________

def print_trainable_parameters(model):
    trainable_params, all_param = 0, 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    print(f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}")

#_______________________________________________________________________

model_path = "stabilityai/stablecode-3b"
model, tokenizer = load_model(model_path)

max_length = get_max_length(model)

# Split the dataset into training and evaluation sets
split_ratio = 0.1
splits = dataset.train_test_split(test_size=split_ratio, seed=seed)
train_dataset = splits['train']
eval_dataset = splits['test']

# Preprocess datasets
train_dataset = preprocess_dataset(tokenizer, max_length, seed, train_dataset)
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, eval_dataset)

#_______________________________________________________________________

# Setup directories upfront
output_dir = "/content/final_checkpoint"
output_merged_dir = "/content/final_checkpoint_merged"

# Create directories if not exists
for dir_path in [output_dir, output_merged_dir]:
    os.makedirs(dir_path, exist_ok=True)

#_______________________________________________________________________

def train(model, tokenizer, train_dataset, eval_dataset, output_dir):
    print("Starting the training process.")

    # Enable gradient checkpointing
    # model.gradient_checkpointing_enable()
    # print("Gradient checkpointing enabled.")

    # Prepare model for k-bit training and locate linear modules
    model = prepare_model_for_kbit_training(model)

    # Freeze all layers except the last one
    for name, param in model.named_parameters():
        if "lm_head" not in name:  # Change "lm_head" according to the name of the final layer in your model
            param.requires_grad = False
        else:
            param.requires_grad = True

    # Output the trainable parameters (this should now be only the last layer)
    print_trainable_parameters(model)

    # Obtain PEFT model
    modules = find_all_linear_names(model)
    print(f"Model prepared for k-bit training. Located linear modules: {modules}")
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    print("PEFT model obtained.")

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=0,
            max_steps=1,
            learning_rate=1e-5,
            fp16=True,
            logging_steps=10,
            output_dir="outputs",
            optim="paged_adamw_8bit",
            eval_strategy="no",
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,
    )
    print("Trainer initialized.")

    model.config.use_cache = False

    # Training
    print(f"BEGINNING TRAINING")
    train_result = trainer.train()
    print(f"Training completed. Metrics: {train_result.metrics}")

    # Evaluate
    print(f"BEGINNING EVALUATION")
    eval_result = trainer.evaluate()
    print(f"Evaluation completed. Results: {eval_result}")

    # Save trained model
    trainer.model.save_pretrained(output_dir)

    # Clear resources
    del model
    del trainer
    torch.cuda.empty_cache()
    print("Training process completed and resources cleaned up.")

#_______________________________________________________________________

# Run training process
train(model, tokenizer, train_dataset, eval_dataset, output_dir)

# Merge weights if needed
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="cuda:0", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
model.save_pretrained(output_merged_dir, safe_serialization=True)
tokenizer.save_pretrained(output_merged_dir)

NVIDIA A100-SXM4-40GB
2.5.1+cu124
True
NOMBRE DE GPU :  2


Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 604
})
Number of prompts: 604
Column names are: ['output', 'input', 'instruction']


,instruction,input,output
0,Custom Types,"let result = x.run(5, 6);",Calls the 'run' method on the 'x' instance wit...
1,,pub fn find_fixed_point(arr: &[i32]) -> Option...,find a fixed point in a given array arguments:...
2,,fn spiral_order(matrix: Vec<Vec<i32>>) -> Vec<...,Returns all elements of a matrix in spiral ord...


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Hello World

### Input:
println!({} days, 31);

### Output:
Prints 31 days using {} as a placeholder for the value (31).

### End


config.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)},
Input predictions: [[ 426  253 1650 ...    0    0    0]
 [ 426  253 1650 ...    0    0    0]
 [ 426  253 1650 ...    0    0    0]
 ...
 [ 426  253 1650 ...    0    0    0]
 [ 426  253 1650 ...    0    0    0]
 [ 426  253 1650 ...    0    0    0]],
Input references: [[30003   310   271 ...  -100  -100  -100]
 [30003   310   271 ...  -100  -100  -100]
 [30003   310   271 ...  -100  -100  -100]
 ...
 [30003   310   271 ...  -100  -100  -100]
 [30003   310   271 ...  -100  -100  -100]
 [30003   310   271 ...  -100  -100  -100]]

In [ ]:
from datasets import load_dataset,Dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

import numpy as np
import random
import pandas as pd
from IPython.display import display
import evaluate

#_______________________________________________________________________

seed = 69
set_seed(seed)
n_gpus = torch.cuda.device_count()
print(torch.__version__)
print(torch.cuda.is_available())
print("NOMBRE DE GPU : ", n_gpus)

accuracy_metric = evaluate.load("accuracy")

#_______________________________________________________________________

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

#_______________________________________________________________________

# Charger le modèle et le tokenizer
def load_model(model_path):
    # Charger le modèle depuis Hugging Face
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",  # Répartir automatiquement sur les GPU ou le CPU
        torch_dtype="auto"  # Adapter automatiquement au type de données
    )
    # Charger le tokenizer correspondant
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

#_______________________________________________________________________

dataset=pd.read_json('/content/data_code.json')

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.columns}')

nb_samples = 3
random_indices = random.sample(range(len(dataset)), nb_samples)
samples = []
for idx in random_indices:
    sample = dataset.iloc[idx]
    sample_data = {
        'instruction': sample['instruction'],
        'input': sample['input'],
        'output': sample['output']
    }
    samples.append(sample_data)
df = pd.DataFrame(samples)
display(df)

#_______________________________________________________________________

def create_prompt_formats(sample):
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "### Input:"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    # Vérification du type de sample pour éviter l'erreur
    if isinstance(sample, str):  # Si sample est une chaîne
        return sample  # Retourner directement la chaîne ou gérer ce cas selon vos besoins

    # Si sample est une Series Pandas
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]
    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt  # Ajouter la clé "text" à la Series Pandas
    return sample

# Appliquer à la première ligne
sample = dataset.iloc[0]
print(create_prompt_formats(sample)["text"])



#_______________________________________________________________________

def get_max_length(model):
    conf = model.config
    max_length = getattr(conf, 'n_positions', None) or getattr(conf, 'max_position_embeddings', None) or getattr(conf, 'seq_length', None) or 1024
    return max_length

#_______________________________________________________________________

def preprocess_batch(batch, max_length, tokenizer, batched=False):
    # Vérifiez si batch est une chaîne de caractères
    if isinstance(batch, str):
        # Si c'est une chaîne de caractères, encapsulez-la dans une liste
        texts = [batch]
    # Vérifiez si batch est une liste de chaînes de caractères
    elif isinstance(batch, list) and isinstance(batch[0], str):
        texts = batch
    # Vérifiez si batch est un dictionnaire (ce qui devrait être le cas dans un Dataset)
    elif isinstance(batch, dict):
        texts = batch["text"]
    else:
        # Si le type de batch ne correspond pas, lever une erreur
        raise TypeError(f"Expected batch to be a string, list, or dict, but got {type(batch)}")

    # Tokenisation des textes
    inputs = tokenizer(texts, max_length=max_length, truncation=True)
    return inputs

#_______________________________________________________________________

def preprocess_dataset(tokenizer, max_length, seed, dataset):
    print("Preprocessing dataset...")

    # Appliquer la fonction create_prompt_formats
    dataset = dataset.apply(create_prompt_formats, axis=1)

    # Prétraiter les lots de données
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset["input_ids"] = dataset["text"].apply(lambda x: _preprocessing_function({"text": x})["input_ids"])

    # Filtrer les échantillons dont la longueur est inférieure à max_length
    dataset = dataset[dataset["input_ids"].apply(len) < max_length]

    # Mélanger les données
    dataset = dataset.sample(frac=1, random_state=seed).reset_index(drop=True)

    return dataset

#_______________________________________________________________________

def create_peft_config(modules):
    return LoraConfig(
        r=16,
        lora_alpha=64,
        target_modules=modules,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
    )

#_______________________________________________________________________

def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

#_______________________________________________________________________

def print_trainable_parameters(model):
    trainable_params, all_param = 0, 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    print(f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}")

#_______________________________________________________________________

model_path = "stabilityai/stablecode-3b"
model, tokenizer = load_model(model_path)

max_length = get_max_length(model)

# Split the dataset into training and evaluation sets
split_ratio = 0.3
train_dataset, eval_dataset = train_test_split(dataset, test_size=split_ratio, random_state=42)
'''train_dataset = splits['train']
eval_dataset = splits['test']'''

# Preprocess datasets
train_dataset = preprocess_dataset(tokenizer, max_length, seed, train_dataset)
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, eval_dataset)

train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

#_______________________________________________________________________
tokenizer.pad_token = tokenizer.eos_token
def train(model, tokenizer, train_dataset, eval_dataset, output_dir):
    # Assurez-vous que train_dataset et eval_dataset sont des objets de type Dataset
    if isinstance(train_dataset, pd.DataFrame):
        train_dataset = Dataset.from_pandas(train_dataset)
    if isinstance(eval_dataset, pd.DataFrame):
        eval_dataset = Dataset.from_pandas(eval_dataset)

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    modules = find_all_linear_names(model)
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    print_trainable_parameters(model)

    training_args = TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=500,  # Ajustez selon la taille du dataset
        learning_rate=2e-4,
        fp16=False,
        logging_steps=10,
        output_dir=output_dir,
        optim="paged_adamw_8bit",
        eval_strategy="epoch",  # Enable evaluation
    )

    trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        args=training_args,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,  # Metric computation function
    )
    model.config.use_cache = False

    # Entraînement
    print("Training...")
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    print(metrics)

    # Évaluation
    print("Evaluating...")
    eval_result = trainer.evaluate()
    print(eval_result)

    # Sauvegarde du modèle
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)
    del model
    del trainer
    torch.cuda.empty_cache()

#_______________________________________________________________________

output_dir = "/content/final_checkpoint"
train(model, tokenizer, train_dataset, eval_dataset, output_dir)

# Merge weights
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
output_merged_dir = "E:/AI/projets/LLM project efrei/llm_project_M2/results/final_checkpoint_merged"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.save_pretrained(output_merged_dir)

2.5.1+cu121
False
NOMBRE DE GPU :  0
Number of prompts: 604
Column names are: Index(['output', 'input', 'instruction'], dtype='object')


,instruction,input,output
0,Custom Types,"let result = x.run(5, 6);",Calls the 'run' method on the 'x' instance wit...
1,,pub fn find_fixed_point(arr: &[i32]) -> Option...,find a fixed point in a given array arguments:...
2,,fn spiral_order(matrix: Vec<Vec<i32>>) -> Vec<...,Returns all elements of a matrix in spiral ord...


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Hello World

### Input:
println!({} days, 31);

### Output:
Prints 31 days using {} as a placeholder for the value (31).

### End


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Preprocessing dataset...
Preprocessing dataset...


max_steps is given, it will override any value given in num_train_epochs


all params: 2,820,477,952 || trainable params: 25,034,752 || trainable%: 0.8876067257412122
Training...


NameError: name 'str2optimizer8bit_blockwise' is not defined